In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
import matplotlib.pyplot as plt
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/WeatherPy_challenge.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description,Rain Inches (last 3 hrs),Snow Inches (last 3 hrs),Date
0,0,Thompson,55.74,-97.86,14.00,66,90,5.82,CA,overcast clouds,0.00,0.0,2020-02-23 18:40:10
1,1,Busselton,-33.65,115.33,68.00,44,93,15.14,AU,light rain,0.69,0.0,2020-02-23 18:40:44
2,2,Rio Grande,-32.03,-52.10,78.80,47,20,17.22,BR,few clouds,0.00,0.0,2020-02-23 18:49:01
3,3,Rikitea,-23.12,-134.97,78.37,74,2,8.25,PF,light rain,0.44,0.0,2020-02-23 18:40:23
4,4,Mataura,-46.19,168.86,51.01,92,93,3.00,NZ,overcast clouds,0.00,0.0,2020-02-23 18:40:31


In [3]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
Rain = str(input("Do you want it to be raining? (yes/no) "))
Snow = str(input("Do you want it to be snowing? (yes/no) "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90
Do you want it to be raining? (yes/no) yes
Do you want it to be snowing? (yes/no) yes


In [4]:
# filter the dataset to find the cities
preferred_cities_df=city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]

if Rain=="no" and Snow=="no":
    preferred_cities_df=preferred_cities_df.loc[(preferred_cities_df["Rain inches(last 3 hrs)"]<=0) & (preferred_cities_df["Snow inches(last 3 hrs)"]<=0)]                                                                          

elif Rain=="no":
    preferred_cities_df=preferred_cities_df.loc[(preferred_cities_df["Rain inches(last 3 hrs)"]<=0),:]
                                                 
elif Snow=="no":
    preferred_cities_df=preferred_cities_df.loc[(preferred_cities_df["Snow inches(last 3 hrs)"]<=0),:]                                             

else:
    preferred_cities_df

preferred_cities_df.info()                                       

<class 'pandas.core.frame.DataFrame'>
Int64Index: 230 entries, 2 to 555
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   City_ID                   230 non-null    int64  
 1   City                      230 non-null    object 
 2   Lat                       230 non-null    float64
 3   Lng                       230 non-null    float64
 4   Max Temp                  230 non-null    float64
 5   Humidity                  230 non-null    int64  
 6   Cloudiness                230 non-null    int64  
 7   Wind Speed                230 non-null    float64
 8   Country                   229 non-null    object 
 9   Current Description       230 non-null    object 
 10  Rain Inches (last 3 hrs)  230 non-null    float64
 11  Snow Inches (last 3 hrs)  230 non-null    float64
 12  Date                      230 non-null    object 
dtypes: float64(6), int64(3), object(4)
memory usage: 25.2+ KB


In [5]:
# drop null rows
preferred_cities_df=preferred_cities_df.dropna()

In [6]:
preferred_cities_df.count()

City_ID                     229
City                        229
Lat                         229
Lng                         229
Max Temp                    229
Humidity                    229
Cloudiness                  229
Wind Speed                  229
Country                     229
Current Description         229
Rain Inches (last 3 hrs)    229
Snow Inches (last 3 hrs)    229
Date                        229
dtype: int64

In [7]:
# Print results rain and snow count
rain_count=preferred_cities_df.loc[preferred_cities_df["Rain Inches (last 3 hrs)"]>0, ["Rain Inches (last 3 hrs)"]].count()
snow_count=preferred_cities_df.loc[preferred_cities_df["Snow Inches (last 3 hrs)"]>0, ["Snow Inches (last 3 hrs)"]].count()
print(f'rain count:{rain_count},snow count:{snow_count}')

rain count:Rain Inches (last 3 hrs)    48
dtype: int64,snow count:Snow Inches (last 3 hrs)    0
dtype: int64


In [8]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [9]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng","Current Description"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
2,Rio Grande,BR,78.80,-32.03,-52.10,few clouds,
3,Rikitea,PF,78.37,-23.12,-134.97,light rain,
8,Fare,PF,82.72,-16.70,-151.02,scattered clouds,
10,Ixtapa,MX,80.60,20.70,-105.20,scattered clouds,
13,Cayenne,GF,84.20,4.93,-52.33,broken clouds,
17,Kavieng,PG,84.34,-2.57,150.80,scattered clouds,
19,Puerto Ayora,EC,84.20,-0.74,-90.35,few clouds,
21,Hermanus,ZA,73.00,-34.42,19.23,clear sky,
22,Castro,BR,79.45,-24.79,-50.01,overcast clouds,
23,Abeche,TD,79.45,13.83,20.83,clear sky,


In [10]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [11]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    # Grab the first hotel from the results and store the name.
    try:
       hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
     print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [12]:
# Set the parameters to search for a hotel in Paris.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522"}
# Use base URL to search for hotels in Paris.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
hotels = requests.get(base_url, params=params).json()

In [13]:
hotels

{'html_attributions': [],
 'next_page_token': 'CqQCHgEAAIworRKfnTDu1tpZMkLvCRlJv8W86uqgUuUko8ZGBN8SCJUukU8ej7RwY-DbIE2bBIDaiKmYxuoFSHMks0qIgITFsX_I6psbKqTAhvG1l4lc6d7Tfq35WIZ-NkutrkWFfIUJm7OGfB5m9gkT_LUpUM4Qc-yoDRT79ekQZGMKBWq2SHkhiyTEjf6_UbFUPXJPSROFG3NlAE9TBJSNvbtuQWiEs7txJSeSXNqGPVGD0hO7jjpa-v5ySUvp7_IKn04Keo5Eq5lQCgE9HzdYxvndAaIX5GiQAPzVIElmHhn1j7eHh98te6BYpEfLTQYtiH1G-oWD69v4IzunIJBazh38Z1HSyVvXTM4WDtNhw0mJXTfUJR5B42bFeZXAg3P2TYDQQxIQ1qqQS5PJA89feOeTjXjAiBoUc6716nsBk5ybf1jPVwuFaFNYRrY',
 'results': [{'geometry': {'location': {'lat': 48.8581126, 'lng': 2.3529277},
    'viewport': {'northeast': {'lat': 48.8594402302915,
      'lng': 2.354348530291502},
     'southwest': {'lat': 48.8567422697085, 'lng': 2.351650569708498}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png',
   'id': '7fa47cb61d7345cecc3b802d0e04fcff2b3d5ecb',
   'name': 'Hôtel Duo',
   'opening_hours': {'open_now': True},
   'photos': [{'height': 3840,
     'html_attributions': ['<a href=

In [14]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
2,Rio Grande,BR,78.80,-32.03,-52.10,few clouds,Swan Express Rio Grande
3,Rikitea,PF,78.37,-23.12,-134.97,light rain,Pension Maro'i
8,Fare,PF,82.72,-16.70,-151.02,scattered clouds,Maitai Lapita Village Huahine
10,Ixtapa,MX,80.60,20.70,-105.20,scattered clouds,Container Inn Puerto Vallarta
13,Cayenne,GF,84.20,4.93,-52.33,broken clouds,Hôtel Le Dronmi
17,Kavieng,PG,84.34,-2.57,150.80,scattered clouds,Nusa Island Retreat
19,Puerto Ayora,EC,84.20,-0.74,-90.35,few clouds,Finch Bay Galapagos Hotel
21,Hermanus,ZA,73.00,-34.42,19.23,clear sky,Misty Waves Boutique Hotel
22,Castro,BR,79.45,-24.79,-50.01,overcast clouds,CHACARA BAILLY
23,Abeche,TD,79.45,13.83,20.83,clear sky,Assaleh Hamdan Haroune


In [15]:
new_col_ord = ["City", "Country","Max Temp","Current Description", "Lat","Lng", "Hotel Name"]
hotel_df = hotel_df[new_col_ord]
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Rio Grande,BR,78.80,few clouds,-32.03,-52.10,Swan Express Rio Grande
3,Rikitea,PF,78.37,light rain,-23.12,-134.97,Pension Maro'i
8,Fare,PF,82.72,scattered clouds,-16.70,-151.02,Maitai Lapita Village Huahine
10,Ixtapa,MX,80.60,scattered clouds,20.70,-105.20,Container Inn Puerto Vallarta
13,Cayenne,GF,84.20,broken clouds,4.93,-52.33,Hôtel Le Dronmi
17,Kavieng,PG,84.34,scattered clouds,-2.57,150.80,Nusa Island Retreat
19,Puerto Ayora,EC,84.20,few clouds,-0.74,-90.35,Finch Bay Galapagos Hotel
21,Hermanus,ZA,73.00,clear sky,-34.42,19.23,Misty Waves Boutique Hotel
22,Castro,BR,79.45,overcast clouds,-24.79,-50.01,CHACARA BAILLY
23,Abeche,TD,79.45,clear sky,13.83,20.83,Assaleh Hamdan Haroune


In [16]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

In [17]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [21]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig



Figure(layout=FigureLayout(height='420px'))

In [19]:
#Save the image
plt.savefig("weather_data/WeatherPy_vacation_map.png")

<Figure size 432x288 with 0 Axes>

In [20]:
# Create the output file (CSV).
output_data_file = "weather_data/WeatherPy_vacation.csv"
hotel_df.to_csv(output_data_file, index_label="City_ID")